In [35]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
evDF = spark.read.option('header','true').option('inferSchema','true').option('delimiter',';').csv('datasets/ev-fiyatlari.csv')

#evDF.printSchema()
vec = VectorAssembler(inputCols=['oda','salon','alan','yas'], outputCol='features')
evDF = vec.transform(evDF)

evDF = evDF.withColumnRenamed('fiyat','label')
#evDF.show()

#Model Training satırına kadar olan kısıma Data Cleansing, Feature Selection ve Feature Enginering 
lr = LinearRegression()
modelLR = lr.fit(evDF) # Model Training (Makine Öğrenmesi tam bu satırda yapılır)
print('bitti')
print("Kat sayılar (Coefficients) : ",model.coefficients)
print("Sabit sayı  (Intercept) : ",model.intercept)

#Modeli (Fonksiyon) hesapladıktan sonra modelin başarısını arttırmak için Parametre ince ayar (Hyper Parameter Tunning) yapmak gerekir

bitti
Kat sayılar (Coefficients) :  [96677.66068344089,0.0,881.5784548802857,-8809.21275403053]
Sabit sayı  (Intercept) :  98322.38079593662


In [24]:
#Linear Regression modeli  (Linear Model)
def evFiyatiNedir(oda,salon,alan,yas) :
    fiyat = 96677 * oda +  0 * salon + 881 * alan + -8809 * yas + 98322
    return fiyat

In [25]:
a = evFiyatiNedir(2,1,190,1) # Prediction
print("Yeni evin fiyatı ",a)

Yeni evin fiyatı  450257


In [26]:
def hastaMi(t1,t2,t3,t4,t5,t6,t7,t8):
    return 0

#False (Accuracy)

#False Positive (Precision değerine bakmak gerekir)
#False Negative (Recall değerine bakmak gerekir)

In [27]:
#Karar ağacı (Decision tree) Model  (Non-Linear Model, Tree-Based)
def evFiyatiNedir2(oda,salon,alan,yas) :
    if oda <= 3:
        if salon <=1:
            if alan > 100:
                return 380000
            else:
                return 270000
        else:
            if oda >= 3.5:
                return 480000
            else:
                return 400000
    else:
        if yas < 5:
            return 800000
            
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor()
modelDT = dt.fit(evDF) #Yukarıdaki fonksiyona benzer bir fonksiyon bulmaya çalışıyoruz
print('Bitti')     

Bitti


In [28]:
print(modelDT.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_82c13989ac3b, depth=3, numNodes=9, numFeatures=4
  If (feature 0 <= 2.5)
   If (feature 0 <= 1.5)
    Predict: 195000.0
   Else (feature 0 > 1.5)
    If (feature 2 <= 95.0)
     Predict: 280000.0
    Else (feature 2 > 95.0)
     Predict: 330000.0
  Else (feature 0 > 2.5)
   If (feature 2 <= 190.0)
    Predict: 450000.0
   Else (feature 2 > 190.0)
    Predict: 600000.0



In [34]:
yenilerDF = spark.read.option('header','true')\
        .option('inferSchema','true')\
        .option('delimiter',';').csv('datasets/yeni-evler.csv')

vec = VectorAssembler(inputCols=['oda','salon','alan','yas'],outputCol='features')
yenilerDF = vec.transform(yenilerDF)

yenilerDF.show()
lrSonuclarDF = modelLR.transform(yenilerDF)

lrSonuclarDF.show()

+---+-----+----+---+-------------------+
|oda|salon|alan|yas|           features|
+---+-----+----+---+-------------------+
|  1|    1| 115|  5|[1.0,1.0,115.0,5.0]|
|  2|    1|  77|  2| [2.0,1.0,77.0,2.0]|
|  2|    1|  80|  3| [2.0,1.0,80.0,3.0]|
|  3|    1| 180|  5|[3.0,1.0,180.0,5.0]|
|  1|    1|  75|  0| [1.0,1.0,75.0,0.0]|
|  2|    1| 122|  1|[2.0,1.0,122.0,1.0]|
+---+-----+----+---+-------------------+

+---+-----+----+---+-------------------+------------------+
|oda|salon|alan|yas|           features|        prediction|
+---+-----+----+---+-------------------+------------------+
|  1|    1| 115|  5|[1.0,1.0,115.0,5.0]| 252335.5000204577|
|  2|    1|  77|  2| [2.0,1.0,77.0,2.0]|341940.81768053933|
|  2|    1|  80|  3| [2.0,1.0,80.0,3.0]| 335776.3402911497|
|  3|    1| 180|  5|[3.0,1.0,180.0,5.0]| 502993.4209545581|
|  1|    1|  75|  0| [1.0,1.0,75.0,0.0]|261118.42559539893|
|  2|    1| 122|  1|[2.0,1.0,122.0,1.0]| 390421.0609041827|
+---+-----+----+---+-------------------+---------

In [33]:
dtSonuclarDF = modelDT.transform(yenilerDF)
dtSonuclarDF.show()

+---+-----+----+---+-------------------+----------+
|oda|salon|alan|yas|           features|prediction|
+---+-----+----+---+-------------------+----------+
|  1|    1| 115|  5|[1.0,1.0,115.0,5.0]|  195000.0|
|  2|    1|  77|  2| [2.0,1.0,77.0,2.0]|  280000.0|
|  2|    1|  80|  3| [2.0,1.0,80.0,3.0]|  280000.0|
|  3|    1| 180|  5|[3.0,1.0,180.0,5.0]|  450000.0|
|  1|    1|  75|  0| [1.0,1.0,75.0,0.0]|  195000.0|
|  2|    1| 122|  1|[2.0,1.0,122.0,1.0]|  330000.0|
+---+-----+----+---+-------------------+----------+

